In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
from functions import RNN_generator,createGraphRNN,init_state_update,RNN_forecast_Repeat,loss_func
import cPickle

In [2]:
discreteList = ['dayOfWeek','payDay','month','earthquake','type','locale','locale_name','transferred','onpromotion']

In [3]:
batch_size = 100
seq_len = 16 # same as test set for convenience
learningRate = 1e-4
epoch = 30
keep_prob = 0.75
n_layers = 2
grad_clip = 5.0
bucketSize = 10000

In [4]:
cardinalitys_X = [55, 4001, 34, 337, 2, 23, 17, 6, 18]
cardinalitys_T = [7, 2, 13, 2, 7, 4, 25, 2, 3]
dimentions_X = [2, 20, 1, 2, 1, 1, 1, 1, 1]
dimentions_T = [1, 1, 1, 1, 1, 1, 1, 1, 1]

In [5]:
dX = sum(dimentions_X)
dT = sum(dimentions_T)
d = dX + dT + 2 # 2 for two cont variables

*** Build Computation Graph ***

In [16]:
inputs,train_op,cost,saver,yhat,state = createGraphRNN(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,\
                                                    dX,d,keep_prob,n_layers,grad_clip)

** Training **

In [17]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [18]:
monitor = 40000

In [9]:
prefix = 'train'
y_np = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [10]:
prefix = 'val_SI'
y_np_val = np.loadtxt(prefix+'_Y',dtype=np.float32, delimiter=",") 
weight_np_val = np.loadtxt(prefix+'_Weight',dtype=np.float32, delimiter=",") 
Con_np_val = np.loadtxt(prefix+'_Con',dtype=np.float32, delimiter=",") 
X_np_val = np.loadtxt(prefix+'_X',dtype=np.int32,delimiter=",") 
Count_np_val = np.loadtxt(prefix+'_Count',dtype=np.int32,delimiter=",") 
Dis_np_val = [np.loadtxt(prefix+'_Dis'+str(j),dtype=np.int32,delimiter=",")  for j in range(len(discreteList))]

In [11]:
index = np.loadtxt('Index_val',dtype=np.int32,delimiter=",") 

In [19]:
init_state = tuple([np.zeros((batch_size,d),dtype=np.float32) for i in range(n_layers)])
for i in range(epoch):
    cost_train = .0
    weight_train = .0
    for j,X_nps in enumerate(RNN_generator(y_np, weight_np,Con_np,Dis_np,X_np,Count_np,\
                              batch_size,seq_len,bucketSize,downSample=0.9)):
        _,cost_np,init_state = sess.run([train_op,cost,state],\
                             dict(zip(inputs,X_nps+[learningRate,init_state])))
        cost_train += cost_np
        weight_train = weight_train + np.sum(X_nps[1])
        
        if j%monitor == 0:
            saver.save(sess,'RNN_fillin_01')
            inputs,train_op,cost,saver,yhat,state = createGraphRNN(None,1,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,\
                                                    dX,d,keep_prob,n_layers,grad_clip)
            sess = tf.InteractiveSession()
            sess.run(tf.global_variables_initializer())
            saver.restore(sess,'RNN_fillin_01')
            init_tot_list = init_state_update(sess,inputs,state,batch_size*10,d,n_layers,\
                                  y_np[index],Con_np[index],X_np[index],Count_np[index],\
                                  [dis[index] for dis in Dis_np])
            y_val_hat = RNN_forecast_Repeat(10,sess,inputs,state,yhat,batch_size*10,n_layers,\
                                            np.expand_dims(y_np[index,Count_np[index]-1],-1),\
                                            Con_np_val,X_np_val,Dis_np_val,init_tot_list)
            print "Train loss:{},Val Loss:{}".format(np.sqrt(seq_len*batch_size*cost_train/weight_train),\
                                                    loss_func(weight_np_val[:,np.newaxis],y_val_hat,y_np_val))
            cost_train = .0
            weight_train = .0
            inputs,train_op,cost,saver,yhat,state = createGraphRNN(batch_size,seq_len,cardinalitys_X,cardinalitys_T,\
                                                    dimentions_X,dimentions_T,\
                                                    dX,d,keep_prob,n_layers,grad_clip)
            sess = tf.InteractiveSession()
            sess.run(tf.global_variables_initializer())
            saver.restore(sess,'RNN_fillin_01')           

INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:1.08884670045,Val Loss:5.6944861412
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:1.09453279027,Val Loss:4.60185289383
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:0.856183945778,Val Loss:4.33840370178
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:0.932031309756,Val Loss:4.22241544724
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:0.777827130165,Val Loss:4.13084363937
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters from RNN_fillin_01
Train loss:1.0011212277,Val Loss:4.05689525604
INFO:tensorflow:Restoring parameters from RNN_fillin_01
INFO:tensorflow:Restoring parameters fr